In [1]:
from brt.router.utils import generate_dst_indices, generate_src_indices
import torch

dst_num = 4
gates = torch.randn((4, 4)).cuda()
topk_indices = torch.topk(gates, k=1, dim=1).indices

route_indices = (
    torch.zeros(
        gates.size(0),
        dst_num,
        dtype=torch.int64,
        device=gates.device,
    )
    .scatter_(1, topk_indices, 1)
    .cuda()
)

supported_capacities = torch.Tensor([1, 2, 4, 8, 16]).int().cuda()

local_indices, dst_loads = generate_dst_indices(
    route_indices, supported_capacities
)
print(local_indices)
print(dst_loads)




: 

: 

In [ ]:
from brt._C.router import route_with_local_indices
import torch

in_data = torch.randn(4, 4, 4, dtype=torch.float32, requires_grad=True).cuda()

total_load = torch.sum(dst_loads).item()
out_data = torch.zeros((total_load, 4), dtype=in_data.dtype).cuda()

new_gates = torch.ones_like(gates, dtype=torch.float32).cuda()

out_data = route_with_local_indices(
    in_data,
    local_indices,
    dst_loads,
)

print(in_data)
print(out_data)


In [ ]:
from brt._C.router import route_back_with_local_indices

final_data = route_back_with_local_indices(out_data, local_indices, dst_loads)

print(final_data)
print(torch.allclose(final_data, in_data))
print(final_data.shape)